# TDC ADMET, Caco-2_Wang Submission

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd

# cheminformatics
import rdkit.Chem
from rdkit.Chem import Descriptors
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# logging
import tqdm

# data preprocessing
import sklearn.impute
import sklearn.preprocessing

# modeling
import sklearn.ensemble
from sklearn.model_selection import ParameterGrid

# metrics
import sklearn.metrics

from tdc.single_pred import ADME


In [2]:
data = ADME(name = 'Caco2_Wang')
split = data.get_split()

Found local copy...
Loading...
Done!


In [3]:
def add_descriptor_columns(data: pd.DataFrame) -> pd.DataFrame:
    """
    Use rdkit to get descriptors of each drug in the `data` df.
    Return a Pandas DataFrame with the descriptors as columns in the df and .
    """
    
    # Extract the Drug column
    assert 'Drug' in data.columns, "'Drug' must be a column in the input DataFrame."
    drugs = data['Drug']
    y = data['Y']
    
    # Get the descriptors for each drug
    print("Calculating descriptors...")
    descriptors = []
    for drug, target in tqdm.tqdm(zip(drugs, y)):
        descriptor = Descriptors.CalcMolDescriptors(
            rdkit.Chem.MolFromSmiles(drug)
        )
        descriptor['Drug'] = drug
        descriptor['Y'] = target
        descriptors.append(descriptor)

    # Make a dataframe for the descriptors
    df = pd.DataFrame(descriptors)

    return df

def preprocess_data(
    data: pd.DataFrame, 
    imputer=sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='mean'),
    fit_imputer=True,
    scaler_X=sklearn.preprocessing.RobustScaler(),
    scaler_y=sklearn.preprocessing.RobustScaler(),
    fit_scaler=True
):
    """
    Imputes missing values.
    Scales feature data.

    Returns a tuple X, y of scaled feature data and target data.
    """

    col_array = np.array(data.columns)

    # extract just the feature data
    X = data[col_array[~np.isin(col_array, ['Drug_ID', 'Drug', 'Y'])]].to_numpy()
    
    # extract the target data
    y = np.array(data['Y']).reshape(-1,1)
    
    # impute missing data
    if imputer is not None:
        if fit_imputer:
            X = imputer.fit_transform(X)
        else:
            X = imputer.transform(X)

    # scale the feature data
    if scaler_X is not None:
        if fit_scaler:
            X = scaler_X.fit_transform(X)
            y = scaler_y.fit_transform(y)
        else:
            X = scaler_X.transform(X)
            y = scaler_y.transform(y)



    return X, y, imputer, scaler_X, scaler_y

In [4]:
X_train, y_train, imputer, scaler_X, scaler_y = preprocess_data(
    add_descriptor_columns(split['train'])
)
X_val, y_val, _, _, _ = preprocess_data(
    add_descriptor_columns(split['valid']),
    imputer=imputer, fit_imputer=False,
    scaler_X=scaler_X, scaler_y=scaler_y,
    fit_scaler=False)


Calculating descriptors...


637it [00:07, 83.55it/s] 


Calculating descriptors...


91it [00:00, 94.33it/s]


In [5]:
from tqdm import tqdm
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterGrid, RandomizedSearchCV
import numpy as np
import os
import json
import joblib

# Папка для сохранения модели и параметров
model_dir = "model/Cacao2"
os.makedirs(model_dir, exist_ok=True)

# ------------------ Stage 1: грубый поиск ------------------ #
params_grid_stage1 = params_grid_stage1 = {
    "C": [10 ** i for i in range(-1, 6)],         # [0.1, 1, 10, 100, 1000, 10000, 100000]
    "gamma": [10 ** i for i in range(-7, 1)],     # [1e-7, 1e-6, ..., 1e0]
    "epsilon": [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]  # более широкий диапазон
}


best_score = float('inf')
best_set = {}
best_model = None

for param_set in tqdm(ParameterGrid(params_grid_stage1), desc="Grid Search Stage 1"):
    model = SVR(kernel="rbf", **param_set)
    model.fit(X_train, y_train.ravel())

    y_val_pred_tmp = model.predict(X_val)
    score_MAE = mean_absolute_error(y_val, y_val_pred_tmp)

    if score_MAE < best_score:
        best_score = score_MAE
        best_set = param_set
        best_model = model

print("Лучшие параметры (Stage 1):", best_set)
print("MAE (Stage 1):", best_score)

# ------------------ Stage 2: уточнение ------------------ #
C_best = best_set["C"]
gamma_best = best_set["gamma"]
eps_best = best_set["epsilon"]

params_grid_stage2 = {
    "C": [C_best * f for f in [0.5, 0.8, 1.0, 1.2, 2.0]],
    "gamma": [gamma_best * f for f in [0.5, 0.8, 1.0, 1.2, 2.0]],
    "epsilon": [max(1e-4, eps_best * f) for f in [0.5, 0.8, 1.0, 1.2, 2.0]]
}

for param_set in tqdm(ParameterGrid(params_grid_stage2), desc="Grid Search Stage 2"):
    model = SVR(kernel="rbf", **param_set)
    model.fit(X_train, y_train.ravel())

    y_val_pred_tmp = model.predict(X_val)
    score_MAE = mean_absolute_error(y_val, y_val_pred_tmp)

    if score_MAE < best_score:
        best_score = score_MAE
        best_set = param_set
        best_model = model

print("Лучшие параметры (Stage 2):", best_set)
print("MAE (Stage 2):", best_score)

# ------------------ Stage 3: RandomizedSearchCV ------------------ #
param_distributions = {
    "C": np.logspace(np.log10(best_set["C"] * 0.5), np.log10(best_set["C"] * 2), 100),
    "gamma": np.logspace(np.log10(best_set["gamma"] * 0.5), np.log10(best_set["gamma"] * 2), 100),
    "epsilon": np.linspace(max(1e-4, best_set["epsilon"] * 0.5), best_set["epsilon"] * 2, 100)
}

random_search = RandomizedSearchCV(
    SVR(kernel="rbf"),
    param_distributions=param_distributions,
    n_iter=20,              # 20 случайных комбинаций
    scoring="neg_mean_absolute_error",
    cv=[(np.arange(len(X_train)), np.arange(len(X_val)))],  # имитация train/val split
    random_state=42,
    verbose=1,
    n_jobs=-1
)

random_search.fit(
    np.vstack((X_train, X_val)), 
    np.hstack((y_train.ravel(), y_val.ravel())))


best_model = random_search.best_estimator_
best_set = random_search.best_params_
best_score = -random_search.best_score_

print("Лучшие параметры (Stage 3 RandomizedSearchCV):", best_set)
print("MAE (Stage 3):", best_score)

# ------------------ Сохраняем модель и параметры ------------------ #
model_path = os.path.join(model_dir, "best_model_svm.pkl")
joblib.dump(best_model, model_path)

params_path = os.path.join(model_dir, "best_params.json")
with open(params_path, "w") as f:
    json.dump({
        "best_score": best_score,
        "best_params": best_set
    }, f, indent=4)

print("✅ Лучшая модель сохранена в:", model_path)
print("✅ Параметры сохранены в:", params_path)


Grid Search Stage 1: 100%|██████████| 336/336 [00:41<00:00,  8.19it/s]


Лучшие параметры (Stage 1): {'C': 1000, 'epsilon': 0.1, 'gamma': 0.0001}
MAE (Stage 1): 0.2899285437538299


Grid Search Stage 2: 100%|██████████| 125/125 [00:14<00:00,  8.47it/s]


Лучшие параметры (Stage 2): {'C': 800.0, 'epsilon': 0.08000000000000002, 'gamma': 0.0001}
MAE (Stage 2): 0.286875443153019
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Лучшие параметры (Stage 3 RandomizedSearchCV): {'gamma': 0.00019721892245187355, 'epsilon': 0.09575757575757578, 'C': 1512.8446244837814}
MAE (Stage 3): 0.14170463745811263
✅ Лучшая модель сохранена в: model/Cacao2\best_model_svm.pkl
✅ Параметры сохранены в: model/Cacao2\best_params.json


In [11]:
from tdc.benchmark_group import admet_group
from sklearn.svm import SVR
import numpy as np

group = admet_group(path='data/')
predictions_list = []

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('Caco2_Wang') 
    predictions = {}
    name = benchmark['name']

    # используем весь train_val для обучения
    train_val, test = benchmark['train_val'], benchmark['test']

    print(f"Seed {seed}:")

    # ---------------- Предобработка ---------------- #
    X_train, y_train, imputer, scaler_X, scaler_y = preprocess_data(
        add_descriptor_columns(train_val)
    )
    X_test, y_test, _, _, _ = preprocess_data(
        add_descriptor_columns(test),
        imputer=imputer, fit_imputer=False,
        scaler_X=scaler_X, fit_scaler=False,
        scaler_y=scaler_y
    )

    # ---------------- SVM (SVR) ---------------- #
    # Пример параметров — при необходимости замените или подберите через CV
    svr_model = SVR(**best_set)
    # Входные y в preprocess_data, как в вашем XGBoost-окружении, вероятно, уже масштабированы
    svr_model.fit(X_train, y_train.ravel())

    # ---------------- Предсказания ---------------- #
    y_pred_test_scaled = svr_model.predict(X_test)  # 1D array
    # inverse_transform ожидает 2D, поэтому reshape
    y_pred_test = scaler_y.inverse_transform(
        y_pred_test_scaled.reshape(-1, 1)
    ).reshape(-1)

    # сохраняем по TDC-шаблону
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

# ---------------- Оценка ---------------- #
results = group.evaluate_many(predictions_list)
print(results)


Found local copy...


Seed 1:
Calculating descriptors...


728it [00:08, 90.98it/s] 


Calculating descriptors...


182it [00:02, 89.37it/s] 


...

Seed 5:
Calculating descriptors...


728it [00:08, 90.20it/s] 


Calculating descriptors...


182it [00:02, 82.99it/s]


{'caco2_wang': [0.374, 0.0]}
